# Tutorial on data cleaning. Unegui.mn example

In [2]:
import pandas as pd
import os
os.getcwd()

'D:\\Documents\\python\\repo\\Introduction_Python\\3_Data_table\\code\\data_cleaning_unegui'

In [79]:
# functions
import re
def get_district(x):
    try:
        x = re.findall('(Баянгол|Баянзүрх|Сүхбаатар|Сонгинохайрхан|Хан-Уул|Чингэлтэй|Налайх)', x)[0]
    except:
        pass
    return x

def extract_currency_value(x):
    price = float(re.findall('(\\d+[\\.\\d]*)', x)[0])
    if 'бум' in x and 'сая' not in x:
        return price * 1e3
    else:
        return price 
def extract_numeric_area_values(x):
    x = float(re.findall('(\\d+[\\.\\d]*)', x)[0])
    return x

In [ ]:
# df1 = pd.DataFrame({'id': [1,1,1,2,3,4,5],'value': [10,11,12,15,25,35,45]} )
# df2 = pd.DataFrame({'id': [1,1,1,2,3,4,5],'value2': [13,14,15,15,25,35,45]} )
# df1.merge(df2,on='id',how='left')

In [57]:
df_newa = pd.read_csv('../../data/data_unegui/sales_apartment_unegui_newer_a.csv')
df_newd = pd.read_csv('../../data/data_unegui/sales_apartment_unegui_newer_district.csv')
df_olda = pd.read_csv('../../data/data_unegui/sales_apartment_unegui_older_a.csv')
df_oldd = pd.read_csv('../../data/data_unegui/sales_apartment_unegui_older_district.csv')

In [58]:
df_newa.drop_duplicates(subset=['id'],keep='first',inplace=True)
df_newd.drop_duplicates(subset=['id'],keep='first',inplace=True)
df_olda.drop_duplicates(subset=['id'],keep='first',inplace=True)
df_oldd.drop_duplicates(subset=['id'],keep='first',inplace=True)

In [59]:
print(len(df_newa),len(df_newd))
print(len(df_olda),len(df_oldd))

7656 7656
3534 3534


In [60]:
df_new = df_newa.merge(df_newd,on=['id'],how='left')
df_old = df_olda.merge(df_oldd,on=['id'],how='left')

In [61]:
print(len(df_new),len(df_old))

7656 3534


In [66]:
df_new.head()
df_old.head()
df_new.columns
df_old.columns
df_new.columns == df_old.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [67]:
df = pd.concat([df_new,df_old])

In [76]:
df["Дүүрэг"] = df["loc"].apply(get_district)

0       Баянзүрх
1        Хан-Уул
2        Хан-Уул
3        Хан-Уул
4       Баянзүрх
          ...   
3529     Хан-Уул
3530     Хан-Уул
3531     Хан-Уул
3532     Хан-Уул
3533     Хан-Уул
Name: Дүүрэг, Length: 11190, dtype: object

In [80]:
df.columns

Index(['Title', 'Price', 'Date', 'Time', 'id', 'phone', 'room', 'message',
       'url', 'Шал:', 'Тагт:', 'Ашиглалтанд орсон он:', 'Гараж:', 'Цонх:',
       'Барилгын давхар:', 'Хаалга:', 'Талбай:', 'Хэдэн давхарт:',
       'Лизингээр авах боломж:', 'Цонхны тоо:', 'Барилгын явц:', 'Код:',
       'Байршил:', 'location:', 'Unnamed: 0', 'loc', 'Дүүрэг'],
      dtype='object')

In [81]:
df["Нийт үнэ"] = df["Price"].apply(extract_currency_value)
df["М2"] = df["Талбай:"].apply(extract_numeric_area_values)

In [83]:
df['М2 үнэ']   = df['Нийт үнэ'].astype(float)
mask = df['Нийт үнэ'].astype(float) >= 5 & ~df['М2'].astype(float).isnull() # no price_m2 higher than or equal to 12 mio at the momemt  
df.loc[mask, 'М2 үнэ'] = df.loc[mask, 'Нийт үнэ'].astype(float) / df.loc[mask, 'М2'].astype(float)

In [84]:
df.head()

,Title,Price,Date,Time,id,phone,room,message,url,Шал:,...,Барилгын явц:,Код:,Байршил:,location:,Unnamed: 0,loc,Дүүрэг,Нийт үнэ,М2,М2 үнэ
0,"Бзд 1-р хороо сансар, и-мартын хойно 4 өрөө ор...",300 сая ₮\nүнэ тохирно,8/30/2023,18:42,7589853,8877-9095,4 өрөө,"\nБЗД 1-р хороо Сансар, И-мартын ард 4 өрөө ор...",https://www.unegui.mn/adv/7589853_bzd-1-r-khor...,Паркет,...,Ашиглалтад орсон,NaN,NaN,NaN,NaN,"УБ — Баянзүрх, Сансар",Баянзүрх,300.00,93.21,3.218539
1,Khans vill хотхонд 3 өрөө байр,302.78 сая ₮,9/11/2023,18:13,7606352,8906-7787,3 өрөө,\nKhans vill 3 өрөө орон сууц худалдана.\n\n91...,https://www.unegui.mn/adv/7606352_khans-vill-k...,Паркет,...,Ашиглалтад орсон,NaN,NaN,NaN,NaN,"УБ — Хан-Уул, Хансвилл хотхон",Хан-Уул,302.78,91.00,3.327253
2,"Худ, 18-р хороо удирдлагын академийн урд богды...",303.66 сая ₮,9/12/2023,11:47,7632026,8600-5021,3 өрөө,"\nХУД, Удирдлагын академийн урд Богдын өндөр х...",https://www.unegui.mn/adv/7632026_khud-18-r-kh...,Паркет,...,Ашиглалтад орсон,NaN,NaN,NaN,NaN,"УБ — Хан-Уул, 120 мянгат",Хан-Уул,303.66,109.00,2.785872
3,Богдын өндөр хотхонд 3 өрөө 108м2 орон сууц,303.66 сая ₮,9/12/2023,11:57,7632083,9999-7941,3 өрөө,"\n📍ХУД, Удирдлагын академийн урд Богдын өндөр ...",https://www.unegui.mn/adv/7632083_bogdyn-ondor...,Паркет,...,Ашиглалтад орсон,NaN,NaN,NaN,NaN,"УБ — Хан-Уул, Хан-Уул, Хороо 15",Хан-Уул,303.66,108.00,2.811667
4,Бзд 26 хороо кристал таунд 80мкв 3 өрөө,307 сая ₮\nүнэ тохирно,8/16/2023,10:29,7515443,9009-8850,3 өрөө,\nурагшаа харсан 3 цонхтой маш цэвэрхэн 3 өрөө...,https://www.unegui.mn/adv/7515443_bzd-26-khoro...,Паркет,...,Ашиглалтад орсон,NaN,NaN,NaN,NaN,"УБ — Баянзүрх, Баянзүрх, Хороо 26",Баянзүрх,307.00,80.00,3.837500


In [88]:
df.groupby('Дүүрэг')['М2 үнэ'].mean()

Дүүрэг
Баянгол                                  NaN
Баянзүрх                                 NaN
Баянхонгор, Баянхонгор              0.055172
Дархан-Уул                          0.619001
Дархан-Уул, Дархан-Уул              2.094577
Дорноговь, Дорноговь                0.842404
Дорнод, Дорнод                      1.887939
Налайх                              2.976677
Орхон                               2.772972
Орхон, Орхон                        2.262211
Сонгинохайрхан                           NaN
Сэлэнгэ                             1.260089
Сэлэнгэ, Сэлэнгэ                    2.340824
Сүхбаатар                                NaN
Төв                                 1.241392
Төв, Төв                            1.910729
УБ — Багануур, Багануур, Хороо 2    1.741729
УБ — Багануур, Багануур, Хороо 3    0.023881
Хан-Уул                             2.938124
Хөвсгөл, Хөвсгөл                    1.607143
Чингэлтэй                           3.352248
Өмнөговь, Өмнөговь                  2.830189
Nam

In [94]:
df[df['Дүүрэг'] == 'Сүхбаатар']['М2 үнэ']

inf